In [1]:
import pandas as pd
import numpy as np
import h5py

from tokenizers import ByteLevelBPETokenizer

In [2]:
samples = pd.read_csv('Reviews.csv')

In [3]:
samples['cleaned'] = samples['Review'].str.lower()
samples['cleaned'] = samples['cleaned'].str.replace(r'[^a-z\s\']', ' ', regex=True)
samples['cleaned'] = samples['cleaned'].str.replace(r'^\s+', '', regex=True)
samples['cleaned'] = samples['cleaned'].str.replace(r'\s+$', '', regex=True)
samples['cleaned'] = samples['cleaned'].str.replace(r'\'', '', regex=True)
samples['cleaned'] = samples['cleaned'].str.replace(r'\s+', ' ', regex=True)

In [4]:
tokenizer = ByteLevelBPETokenizer()
tokenizer.train_from_iterator(samples['cleaned'], vocab_size=4096, min_frequency=256)

In [5]:
vocabulary = tokenizer.get_vocab()
len(vocabulary)

4096

In [6]:
samples['tokens'] = samples['cleaned'].apply(lambda x: tokenizer.encode(x).ids)
tokenized_length = samples['tokens'].apply(len).max()
samples['tokens'] = samples['tokens'].apply(lambda x: (([0]*tokenized_length)+x)[-tokenized_length:])

In [7]:
tokenized_length

3674

In [ ]:
x = np.array(samples['tokens'].tolist()).astype(np.uint16)
y = np.array(samples['Sentiment'].tolist()).astype(np.uint8)

In [9]:
file = h5py.File('xy.h5', 'w')

file.create_dataset('x', data=x)
file.create_dataset('y', data=y)

file.close()